In [ ]:
from SinkhornAutoDiff import *

import jax
import jax.numpy as np
import jax.random as random
from functools import partial


import matplotlib.pylab as plt
import ot
import ot.plot
from ot.datasets import make_1D_gauss

import sys
sys.path.insert(0,'../gp/')
from gpax import *

from otax import *

In [ ]:
# https://pythonot.github.io/auto_examples/plot_OT_1D.html

n = 100

x = np.arange(n)
a = np.asarray(make_1D_gauss(n, m=20, s=5))
b = np.asarray(make_1D_gauss(n, m=60, s=10))

M = ot.dist(x.reshape((n, 1)), x.reshape((n, 1)))
# M = sqdist(x)
M = M / M.max()

fig, ax = plt.subplots(1,1,figsize=(5,5))
ax.plot(x, a, 'b', label='Source distribution')
ax.plot(x, b, 'r', label='Target distribution')
ax.legend()
plt.show()

Gs,log = ot.sinkhorn(a,b,M,1e-2,log=True)
ot.plot.plot1D_mat(a,b, Gs, 'Cost matrix M')

In [ ]:


_,_,cost,P = sinkhorn_knopp(a,b,M,1e-2,100)
plt.figure(4, figsize=(5, 5))
ot.plot.plot1D_mat(a, b, P, 'OT matrix Sinkhorn')
plt.show()
